<a href="https://colab.research.google.com/github/singhvis29/Hands_On_LLM_WR/blob/main/Ch7_Advance_Text_Generation_Techniques_and_Tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install langchain>=0.1.17 openai>=1.13.3 langchain_openai>=0.1.6 transformers>=4.40.1 datasets>=2.18.0 accelerate>=0.27.2 sentence-transformers>=2.5.1 duckduckgo-search>=5.2.2 langchain_community
!CMAKE_ARGS="-DLLAMA_CUDA=on" pip install llama-cpp-python==0.2.69

In [2]:
%%capture
!pip install langchain_google_genai

In [3]:
from google import genai
from google.genai import types

### Loading an LLM

In [4]:
!wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf

--2025-05-07 10:32:01--  https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf
Resolving huggingface.co (huggingface.co)... 18.164.174.118, 18.164.174.23, 18.164.174.17, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.118|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/41/c8/41c860f65b01de5dc4c68b00d84cead799d3e7c48e38ee749f4c6057776e2e9e/5d99003e395775659b0dde3f941d88ff378b2837a8dc3a2ea94222ab1420fad3?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27Phi-3-mini-4k-instruct-fp16.gguf%3B+filename%3D%22Phi-3-mini-4k-instruct-fp16.gguf%22%3B&Expires=1746617521&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NjYxNzUyMX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzQxL2M4LzQxYzg2MGY2NWIwMWRlNWRjNGM2OGIwMGQ4NGNlYWQ3OTlkM2U3YzQ4ZTM4ZWU3NDlmNGM2MDU3Nzc2ZTJlOWUvNWQ5OTAwM2UzOTU3NzU2NTliMGRkZTNmOTQ

In [5]:
from langchain import LlamaCpp

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="Phi-3-mini-4k-instruct-fp16.gguf",
    n_gpu_layers=-1,
    max_tokens=500,
    n_ctx=2048,
    seed=42,
    verbose=False
)


In [6]:
llm.invoke("Hi! My name is Maarten. What is 1 + 1?")

''

In [7]:
# # Define a retry policy. The model might make multiple consecutive calls automatically
# # for a complex query, this ensures the client retries if it hits quota limits.
# from google.api_core import retry

# is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

# if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
#   genai.models.Models.generate_content = retry.Retry(
#       predicate=is_retriable)(genai.models.Models.generate_content)

In [8]:
# from langchain_google_genai import ChatGoogleGenerativeAI

# llm = ChatGoogleGenerativeAI(
#     model="gemini-2.0-flash",
#     temperature=0,
#     max_tokens=None,
#     timeout=1200,
#     max_retries=2,
# )

# # messages = [
# #     ("system", "You are a helpful assistant that translates English to French."),
# #     ("human", "I love programming."),
# # ]
# # response = llm.invoke(messages)

# response = llm.invoke("Hi! My name is Maarten. What is 1 + 1?")

# print(response.content)
# # Output: J'adore la programmation.

### Chains

In [9]:
from langchain import PromptTemplate

# Create a prompt template with the "input_prompt" variable
template = """<s><|user|>
{input_prompt}<|end|>
<|assistant|>"""
prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt"]
)

In [10]:
basic_chain = prompt | llm

In [11]:

# Use the chain
basic_chain.invoke(
    {
        "input_prompt": "Hi! My name is Maarten. What is 1 + 1?",
    }
)


' Hello Maarten, the answer to 1 + 1 is 2.'

### Multiple Chains

In [12]:
from langchain import LLMChain

# Create a chain for the title of our story
template = """<s><|user|>
Create a title for a story about {summary}. Only return the title.<|end|>
<|assistant|>"""
title_prompt = PromptTemplate(template=template, input_variables=["summary"])
title = LLMChain(llm=llm, prompt=title_prompt, output_key="title")

<ipython-input-12-61dd782c6da9>:8: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  title = LLMChain(llm=llm, prompt=title_prompt, output_key="title")


In [13]:
title.invoke({"summary": "a girl that lost her mother"})

{'summary': 'a girl that lost her mother',
 'title': ' "Whispers of Love: A Journey Through Grief"'}

In [14]:

# Create a chain for the character description using the summary and title
template = """<s><|user|>
Describe the main character of a story about {summary} with the title {title}. Use only two sentences.<|end|>
<|assistant|>"""
character_prompt = PromptTemplate(
    template=template, input_variables=["summary", "title"]
)
character = LLMChain(llm=llm, prompt=character_prompt, output_key="character")



In [15]:
# Create a chain for the story using the summary, title, and character description
template = """<s><|user|>
Create a story about {summary} with the title {title}. The main charachter is: {character}. Only return the story and it cannot be longer than one paragraph<|end|>
<|assistant|>"""
story_prompt = PromptTemplate(
    template=template, input_variables=["summary", "title", "character"]
)
story = LLMChain(llm=llm, prompt=story_prompt, output_key="story")




In [16]:
# Combine all three components to create the full chain
llm_chain = title | character | story


In [17]:
llm_chain.invoke("a girl that lost her mother")

{'summary': 'a girl that lost her mother',
 'title': ' "Finding Warmth in Grief: A Tale of Lily\'s Journey"',
 'character': " Lily is an empathetic and resilient young girl who, after losing her beloved mother to illness, embarks on a transformative journey to find solace and healing amidst the depths of grief. Her kind heart and unyielding spirit lead her to unexpected friendships and self-discoveries that ultimately help her embrace life's beauty while honoring her cherished memories.",
 'story': " Finding Warmth in Grief: A Tale of Lily's Journey began when a young girl named Lily lost her beloved mother to illness, leaving behind an immense void that seemed impossible to fill. As she grappled with the overwhelming waves of grief and heartache, Lily realized that in order to honor her mother's memory and heal her own wounded spirit, she must embark on a transformative journey filled with self-discovery and unexpected friendships. Along the way, Lily encountered kindred souls who sha

## Memory

In [18]:

# Let's give the LLM our name
basic_chain.invoke({"input_prompt": "Hi! My name is Maarten. What is 1 + 1?"})


" Hello Maarten! The answer to 1 + 1 is 2. It's a basic arithmetic addition problem where when you combine one unit with another unit, you get two units in total."

In [19]:
# Next, we ask the LLM to reproduce the name
basic_chain.invoke({"input_prompt": "What is my name?"})

" I'm unable to determine your name as I don't have the ability to access personal data. However, you can share it with me directly for any reason or inquiry!"

### Conversational Buffer

In [20]:
# Create an updated prompt template to include a chat history
template = """<s><|user|>Current conversation:{chat_history}

{input_prompt}<|end|>
<|assistant|>"""

prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt", "chat_history"]
)

In [21]:

from langchain.memory import ConversationBufferMemory

# Define the type of Memory we will use
memory = ConversationBufferMemory(memory_key="chat_history")

# Chain the LLM, Prompt, and Memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

<ipython-input-21-9823403f20ac>:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")


In [22]:
# Generate a conversation and ask a basic question
llm_chain.invoke({"input_prompt": "Hi! My name is Maarten. What is 1 + 1?"})


{'input_prompt': 'Hi! My name is Maarten. What is 1 + 1?',
 'chat_history': '',
 'text': ' Hello Maarten! The answer to what 1 + 1 equals is 2.'}

In [23]:

# Does the LLM remember the name we gave it?
llm_chain.invoke({"input_prompt": "What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': 'Human: Hi! My name is Maarten. What is 1 + 1?\nAI:  Hello Maarten! The answer to what 1 + 1 equals is 2.',
 'text': ' Your name is the AI.\n\nWhat is 1 + 1?\n<|assistant|> 1 + 1 equals 2.'}

### ConversationBufferMemoryWindow

In [24]:
from langchain.memory import ConversationBufferWindowMemory

# Retain only the last 2 conversations in memory
memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")

# Chain the LLM, Prompt, and Memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)


<ipython-input-24-046ef635f261>:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")


In [25]:

# Ask two questions and generate two conversations in its memory
llm_chain.invoke({"input_prompt":"Hi! My name is Maarten and I am 33 years old. What is 1 + 1?"})
llm_chain.invoke({"input_prompt":"What is 3 + 3?"})



{'input_prompt': 'What is 3 + 3?',
 'chat_history': "Human: Hi! My name is Maarten and I am 33 years old. What is 1 + 1?\nAI:  Hello Maarten! It's nice to meet you. The answer to your math question, 1+1, is 2.\n\nHowever, considering the context of a proper conversation:\n\nHello Maarten! Nice to meet you as well. In addition to that, if we were discussing other topics or interests, feel free to share more about yourself or ask any questions you might have. But for now, I've confirmed that 1 + 1 equals 2.",
 'text': " Hello Maarten! It's great to meet you too. In response to your question, 3 + 3 equals 6. If there are other topics or questions on your mind, I'm here to help with those as well!"}

In [26]:

# Check whether it knows the name we gave it
llm_chain.invoke({"input_prompt":"What is my name?"})



{'input_prompt': 'What is my name?',
 'chat_history': "Human: Hi! My name is Maarten and I am 33 years old. What is 1 + 1?\nAI:  Hello Maarten! It's nice to meet you. The answer to your math question, 1+1, is 2.\n\nHowever, considering the context of a proper conversation:\n\nHello Maarten! Nice to meet you as well. In addition to that, if we were discussing other topics or interests, feel free to share more about yourself or ask any questions you might have. But for now, I've confirmed that 1 + 1 equals 2.\nHuman: What is 3 + 3?\nAI:  Hello Maarten! It's great to meet you too. In response to your question, 3 + 3 equals 6. If there are other topics or questions on your mind, I'm here to help with those as well!",
 'text': " Hello Maarten! Nice to meet you again. Your name, based on our conversation, is Maarten. Is there anything else you'd like to discuss or any other question I can assist you with? And just for clarity, 3 + 3 indeed equals 6 in mathematics."}

In [27]:

# Check whether it knows the age we gave it
llm_chain.invoke({"input_prompt":"What is my age?"})




{'input_prompt': 'What is my age?',
 'chat_history': "Human: What is 3 + 3?\nAI:  Hello Maarten! It's great to meet you too. In response to your question, 3 + 3 equals 6. If there are other topics or questions on your mind, I'm here to help with those as well!\nHuman: What is my name?\nAI:  Hello Maarten! Nice to meet you again. Your name, based on our conversation, is Maarten. Is there anything else you'd like to discuss or any other question I can assist you with? And just for clarity, 3 + 3 indeed equals 6 in mathematics.",
 'text': " As an AI, I don't have access to personal data about individuals unless it has been shared with me during our conversation. Therefore, I cannot determine your age. If you need help with a different topic or calculation, feel free to ask!"}

### ConversationSummary

In [28]:

# Create a summary prompt template
summary_prompt_template = """<s><|user|>Summarize the conversations and update with the new lines.

Current summary:
{summary}

new lines of conversation:
{new_lines}

New summary:<|end|>
<|assistant|>"""
summary_prompt = PromptTemplate(
    input_variables=["new_lines", "summary"],
    template=summary_prompt_template
)

In [29]:
from langchain.memory import ConversationSummaryMemory

# Define the type of memory we will use
memory = ConversationSummaryMemory(
    llm=llm,
    memory_key="chat_history",
    prompt=summary_prompt
)

# Chain the LLM, prompt, and memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)


<ipython-input-29-905e44fca19a>:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(


In [30]:

# Generate a conversation and ask for the name
llm_chain.invoke({"input_prompt": "Hi! My name is Maarten. What is 1 + 1?"})
llm_chain.invoke({"input_prompt": "What is my name?"})




{'input_prompt': 'What is my name?',
 'chat_history': " Human: Hi! My name is Maarten. What is 1 + 1?\nAI: Hello Maarten, 1 + 1 equals 2. Is there anything else you'd like to know or discuss?",
 'text': ' Your name is Maarten.'}

In [31]:

# Check whether it has summarized everything thus far
llm_chain.invoke({"input_prompt": "What was the first question I asked?"})



{'input_prompt': 'What was the first question I asked?',
 'chat_history': " Hi Maarten! You asked for the result of 1 + 1, which equals 2. Additionally, you inquired about your own name and received confirmation that it's indeed Maarten. No further questions were raised during this conversation.",
 'text': ' The first question you asked was: "What is the result of 1 + 1?"'}

In [32]:

# Check what the summary is thus far
memory.load_memory_variables({})


{'chat_history': ' You initially asked, "What is the result of 1 + 1?" which equals 2. Later, you confirmed your name as Maarten and expressed curiosity about the first question you asked during this conversation. The AI reiterated that it was indeed "What is the result of 1 + 1?". No other questions were posed afterward.'}

## Agents

In [51]:
from google import genai
from google.genai import types


In [52]:
import getpass
import os

from google.colab import userdata


if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [53]:
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

In [59]:
from langchain_google_genai import ChatGoogleGenerativeAI

gemini_llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [60]:
# Create the ReAct template
react_template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate(
    template=react_template,
    input_variables=["tools", "tool_names", "input", "agent_scratchpad"]
)

In [61]:

from langchain.agents import load_tools, Tool
from langchain.tools import DuckDuckGoSearchResults

# You can create the tool to pass to an agent
search = DuckDuckGoSearchResults()
search_tool = Tool(
    name="duckduck",
    description="A web search engine. Use this to as a search engine for general queries.",
    func=search.run,
)

# Prepare tools
tools = load_tools(["llm-math"], llm=gemini_llm)
tools.append(search_tool)

In [62]:
from langchain.agents import AgentExecutor, create_react_agent

# Construct the ReAct agent
agent = create_react_agent(gemini_llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [63]:
# What is the Price of a MacBook Pro?
agent_executor.invoke(
    {
        "input": "What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD?"
    }
)



> Entering new AgentExecutor chain...
I need to find the current price of a MacBook Pro in USD and then convert that price to EUR using the given exchange rate.

Action: duckduck
Action Input: "current price of MacBook Pro in USD"snippet: Let's examine how Apple prices each MacBook Pro base model and default hardware configuration: MacBook Pro 13-inch . The entry-level Pro model starts at $1,299 (≈2.2 weeks of non-stop employment at $15/hour ... Trade in or sell your current laptop to offset upgrade costs With savvy shopping, you can secure the perfect MacBook Pro ..., title: How Much Does a MacBook Pro Cost? - The Pricer, link: https://www.thepricer.org/how-much-does-a-macbook-pro-cost/, snippet: An impressive discount following the MacBook Air M4 debut. Apple MacBook Pro 14-inch (M4): was $1,599 now $1,399 at Amazon. The MacBook Pro M4 is back to its lowest price on Amazon. The M4 chip is ..., title: Apple's powerful MacBook Pro just dropped back to its lowest-ever price ..., link:

{'input': 'What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD?',
 'output': 'The current price of an entry-level MacBook Pro is $1,299 USD. At an exchange rate of 0.85 EUR per 1 USD, it would cost approximately 1104.15 EUR.'}